In [0]:
HOST="https://gateway.watsonplatform.net/natural-language-understanding/api"
USERNAME="NO"
PASSWORD="NO"

In [0]:
import requests
import json
import csv
import zlib
from requests.auth import HTTPBasicAuth

In [0]:
reviews = requests.get('http://data.insideairbnb.com/canada/on/toronto/2018-08-08/data/reviews.csv.gz')
reviews = zlib.decompress(reviews.content, 16+zlib.MAX_WBITS).decode('utf8')
reviews = list(csv.DictReader(reviews.splitlines()))

In [0]:
reviews = reviews[:50000]

In [9]:
import random
from collections import defaultdict
from queue import Queue
import threading

#random.seed(100)

sen_positive = defaultdict(float)
sen_negative = defaultdict(float)

p_q = Queue()
n_q = Queue()

w_q = Queue()

def proc():
  sen_positive = defaultdict(float)
  sen_negative = defaultdict(float)
  
  while True:
    review = w_q.get()
    if w_q.qsize() % 500 == 0:
      print(w_q.qsize())
    if review is None:
      p_q.put(sen_positive)
      n_q.put(sen_negative)
      break

    req = {
      "text": review["comments"],
      "features": {
        "sentiment": {
        }
      }
    }

    resp = requests.post(HOST + "/v1/analyze?version=2018-03-19", json=req, auth=HTTPBasicAuth(USERNAME, PASSWORD))

    if not resp.ok and resp.json()['error'] != 'cannot locate keyphrase' and resp.json()['error'] != 'invalid request: content is empty':
      print(resp.text)

    resp = resp.json()

    if 'sentiment' in resp:
      listing_id = review['listing_id']
      sentiment = resp['sentiment']['document']
      if sentiment['label'] == 'positive':
        sen_positive[listing_id] += sentiment['score']
      else:
        sen_negative[listing_id] -= sentiment['score']

    
    w_q.task_done()

N = 1000

threads = []

for review in reviews:
  w_q.put(review)

for i in range(N):
  t = threading.Thread(target=proc)
  t.start()
  threads.append(t)

w_q.join()

for i in range(N):
  w_q.put(None)
for t in threads:
  t.join()

  #print(review['listing_id'])
  #print(req['text'])
  #print(resp)
  #print()

[(k, p, sen_negative[k]) for k, p in sen_positive.items()]

{
  "language": "nl",
  "error": "unsupported text language: nl",
  "code": 400
}
{
  "language": "zh",
  "error": "unsupported text language: zh",
  "code": 400
}
{
  "error": "not enough text for language id",
  "code": 422
}
{
  "error": "not enough text for language id",
  "code": 422
}
49500
{
  "error": "not enough text for language id",
  "code": 422
}
{
  "language": "zh",
  "error": "unsupported text language: zh",
  "code": 400
}
49293
{
  "error": "not enough text for language id",
  "code": 422
}
{
  "language": "zh",
  "error": "unsupported text language: zh",
  "code": 400
}
49000
{
  "error": "not enough text for language id",
  "code": 422
}
{
  "error": "not enough text for language id",
  "code": 422
}
{
  "language": "zh",
  "error": "unsupported text language: zh",
  "code": 400
}
{
  "language": "zh",
  "error": "unsupported text language: zh",
  "code": 400
}
{
  "error": "not enough text for language id",
  "code": 422
}
{
  "language": "ca",
  "error": "unsuppor

[]

In [0]:
p_s = []
n_s = []

try:
  while True:
    p_s.append(p_q.get(False))
except:
  pass

try:
  while True:
    n_s.append(n_q.get(False))
except:
  pass

In [0]:
positive = defaultdict(float)
negative = defaultdict(float)

for pp in p_s:
  for k, v in pp.items():
    positive[k] += v
for nn in n_s:
  for k, v in nn.items():
    negative[k] += v

In [17]:
negative

defaultdict(float,
            {'1006928': 1.4359339000000002,
             '1009656': 1.5747569000000001,
             '1009751': 0.311397,
             '1012527': 0.631918,
             '1015594': 0.0419787,
             '1016125': 0.335886,
             '1022764': 0.0,
             '1026288': 0.448211,
             '1026693': 0.768933,
             '1030061': 0.47078,
             '10314': 0.0,
             '1033151': 0.00659025,
             '1033162': 0.445621,
             '1034430': 0.812617,
             '1036728': 0.0,
             '1037034': 0.314137,
             '1038285': 0.0,
             '1040749': 0.0,
             '1043324': 1.1156323000000001,
             '1045585': 0.0,
             '1053838': 0.9425349999999999,
             '1062690': 0.0459422,
             '1066718': 0.0189943,
             '1080121': 2.0839709,
             '1092407': 0.423894,
             '1094362': 1.1534976000000001,
             '1097660': 1.5137779999999998,
             '1100282': 0.0816

In [0]:
import json
with open('rating.json', 'w') as f:
  json.dump({k: p / (p + negative[k]) * 100 for k, p in positive.items() if p}, f)

In [0]:
from google.colab import files
files.download('rating.json')